In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [2]:
# Get current working directory
import os
cwd = os.getcwd()
print("Current working directory:", cwd)
os.chdir("n:/Incubator2025_ComputationalLifeCourse")

Current working directory: n:\Incubator2025_ComputationalLifeCourse\Scripts\g_comp


In [3]:
import sys
sys.path.append("Scripts/g_comp")  # relative path from your current working directory
import g_comp as gc

In [4]:
# Reload the module (if it's been edited and needs to be reloaded)
import importlib
importlib.reload(gc)

<module 'g_comp' from 'n:\\Incubator2025_ComputationalLifeCourse\\Scripts/g_comp\\g_comp.py'>

In [5]:
def set_seed(seed =42):
    """Set seed for reproducibility across multiple libraries"""
    random.seed(seed)  # Python's built-in random
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch
    torch.cuda.manual_seed_all(seed)  # PyTorch CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(2025)  # Call this at the beginning of your code

In [6]:
## Load the data 
import pandas as pd
df = pd.read_csv("Data/Processed/g_data.csv")
df = df.sort_values(["mergeid", "t_age"]) 

In [7]:
## Check the number of dropped cases 
complete_y_adl_65_75_dic_mergeids = gc.summarize_mergeid_completeness(df, ["y_adl_65_75_dic", "dt_n_years_disease_dic"], "ADL 65–75")

[ADL 65–75]
Original mergeids: 20806
Complete mergeids (no missing values in Y): 14958
Number of unique mergeids dropped: 5848



In [8]:
###############
## Data prep ##
###############

# N = number of individuals, T = number of time points (i.e., 33)
# Generate datasets for pooled and by regime 
T = 33
df_adl_65_75 = gc.get_valid_df(df, complete_y_adl_65_75_dic_mergeids, "ADL 65–75", T)
df_adl_med_65_75 = df_adl_65_75[df_adl_65_75["mod_welfare_regime_mediterranean"] == 1.0]
df_adl_cor_65_75 = df_adl_65_75[df_adl_65_75["mod_welfare_regime_corporatist"] == 1.0]
df_adl_scan_65_75 = df_adl_65_75[df_adl_65_75["mod_welfare_regime_scandinavian"] == 1.0]
 

# Mediterranean 
feature_cols_outcome_med_65_75 = gc.get_feature_cols(df_adl_med_65_75, context="outcome and med")
feature_cols_tv_covar_med_65_75 = gc.get_feature_cols(df_adl_med_65_75, context="tv_covar and med")

# Corporatist
feature_cols_outcome_cor_65_75 = gc.get_feature_cols(df_adl_cor_65_75, context="outcome and cor")
feature_cols_tv_covar_cor_65_75 = gc.get_feature_cols(df_adl_cor_65_75, context="tv_covar and cor")

# Scandinavian
feature_cols_outcome_scan_65_75 = gc.get_feature_cols(df_adl_scan_65_75, context="outcome and scan")
feature_cols_tv_covar_scan_65_75 = gc.get_feature_cols(df_adl_scan_65_75, context="tv_covar and scan")


# Convert to (N, T, D) tensor
# Treatment and covariates from the dataset 
 
# Mediterranean 
N_adl_med_65_75 = df_adl_med_65_75['mergeid'].nunique()
X_adl_med_65_75 = gc.convert_df_to_X(df_adl_med_65_75, feature_cols_outcome_med_65_75, N_adl_med_65_75, T)
X_disease_med_65_75  = gc.convert_df_to_X(df_adl_med_65_75, feature_cols_tv_covar_med_65_75, N_adl_med_65_75, T)

# Corporatist
N_adl_cor_65_75 = df_adl_cor_65_75['mergeid'].nunique()
X_adl_cor_65_75 = gc.convert_df_to_X(df_adl_cor_65_75, feature_cols_outcome_cor_65_75, N_adl_cor_65_75, T)
X_disease_cor_65_75  = gc.convert_df_to_X(df_adl_cor_65_75, feature_cols_tv_covar_cor_65_75, N_adl_cor_65_75, T)

# Scandinavian
N_adl_scan_65_75 = df_adl_scan_65_75['mergeid'].nunique()
X_adl_scan_65_75 = gc.convert_df_to_X(df_adl_scan_65_75, feature_cols_outcome_scan_65_75, N_adl_scan_65_75, T)
X_disease_scan_65_75  = gc.convert_df_to_X(df_adl_scan_65_75, feature_cols_tv_covar_scan_65_75, N_adl_scan_65_75, T)

[ADL 65–75]
Mergeids with complete outcome and 33 unique ages: 14958
[get_feature_cols] Context: outcome and med | Exclude: ['mod_country_italy']
⚠️ dt_n_years_disease_dic IS STILL INCLUDED
[get_feature_cols] Context: tv_covar and med | Exclude: ['mod_country_italy', 'dt_n_years_disease_dic']
✅ dt_n_years_disease_dic successfully excluded
[get_feature_cols] Context: outcome and cor | Exclude: ['mod_country_germany']
⚠️ dt_n_years_disease_dic IS STILL INCLUDED
[get_feature_cols] Context: tv_covar and cor | Exclude: ['mod_country_germany', 'dt_n_years_disease_dic']
✅ dt_n_years_disease_dic successfully excluded
[get_feature_cols] Context: outcome and scan | Exclude: ['mod_country_sweden']
⚠️ dt_n_years_disease_dic IS STILL INCLUDED
[get_feature_cols] Context: tv_covar and scan | Exclude: ['mod_country_sweden', 'dt_n_years_disease_dic']
✅ dt_n_years_disease_dic successfully excluded


In [9]:
###############
## Y and L_t ##
###############

# Binary outcomes  
y_adl_med_65_75_dic = gc.extract_y_tensor(df_adl_med_65_75, "y_adl_65_75_dic")
y_adl_cor_65_75_dic = gc.extract_y_tensor(df_adl_cor_65_75, "y_adl_65_75_dic")
y_adl_scan_65_75_dic = gc.extract_y_tensor(df_adl_scan_65_75, "y_adl_65_75_dic")

# Binary outcomes for tv covar 
# Mediterranean 
y_adl_disease_med_65_75_dic = gc.extract_y_tensor(df_adl_med_65_75, "dt_n_years_disease_dic")
y_adl_disease_cor_65_75_dic = gc.extract_y_tensor(df_adl_cor_65_75, "dt_n_years_disease_dic")
y_adl_disease_scan_65_75_dic = gc.extract_y_tensor(df_adl_scan_65_75, "dt_n_years_disease_dic")

In [10]:
#####################
## \vec{a} and L_0 ##
#####################

# Extract high-level treatment trajectory features with covariates for each X matrix
# Mediterranean
X_med_features_adl_65_75_dic = gc.extract_features(X_adl_med_65_75, feature_cols_outcome_med_65_75)
X_med_features_disease_65_75_dic  = gc.extract_features(X_disease_med_65_75, feature_cols_tv_covar_med_65_75)

# Corporatist
X_cor_features_adl_65_75_dic = gc.extract_features(X_adl_cor_65_75, feature_cols_outcome_cor_65_75)
X_cor_features_disease_65_75_dic  = gc.extract_features(X_disease_cor_65_75, feature_cols_tv_covar_cor_65_75)

# Scandinavian
X_scan_features_adl_65_75_dic = gc.extract_features(X_adl_scan_65_75, feature_cols_outcome_scan_65_75)
X_scan_features_disease_65_75_dic  = gc.extract_features(X_disease_scan_65_75, feature_cols_tv_covar_scan_65_75)



# Feature Names
trt_features_med_adl_65_75_dic   = gc.create_feature_df(X_med_features_adl_65_75_dic, feature_cols_outcome_med_65_75, "adl 65–75")
trt_features_cor_adl_65_75_dic   = gc.create_feature_df(X_cor_features_adl_65_75_dic, feature_cols_outcome_cor_65_75, "adl 65–75")
trt_features_scan_adl_65_75_dic   = gc.create_feature_df(X_scan_features_adl_65_75_dic, feature_cols_outcome_scan_65_75, "adl 65–75")


adl 65–75: Feature dataframe shape = (4706, 48)
adl 65–75: Feature dataframe shape = (7381, 48)
adl 65–75: Feature dataframe shape = (2871, 48)


In [11]:
#############################
## Counterfactual analysis ## 
#############################

import pandas as pd
# Load CSV
medoid_df = pd.read_csv("Data/Processed/medoid_seq_data.csv")

# Sort by cluster and t_age
medoid_df = medoid_df.sort_values(["cluster", "t_age"])

# Reorder columns so cluster is first
cols = ["cluster"] + [col for col in medoid_df.columns if col != "cluster"]
medoid_df = medoid_df[cols]

N_medoid = medoid_df['cluster'].nunique()
X_medoid_adl_65_75_dic   = gc.convert_df_to_X(medoid_df, feature_cols_outcome_med_65_75, N_medoid, T)
X_medoid_disease_65_75_dic  = gc.convert_df_to_X(medoid_df, feature_cols_tv_covar_med_65_75, N_medoid, T)
X_medoid_features_65_75_dic   = gc.extract_treatment_features_from_medoid(X_medoid_adl_65_75_dic, feature_cols_outcome_med_65_75) 

In [12]:
#######################
## Load Final models ##
#######################
from joblib import load
 
# Med
y_adl_disease_med_super_learner = load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_disease_65-75_(med)_super_learner.pkl")
y_adl_med_super_learner = load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_65-75_(med)_super_learner.pkl")

# Cor  
y_adl_disease_cor_super_learner = load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_disease_65-75_(cor)_super_learner.pkl")
y_adl_cor_super_learner =  load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_65-75_(cor)_super_learner.pkl")

# Scan 
y_adl_disease_scan_super_learner = load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_disease_65-75_(scan)_super_learner.pkl")
y_adl_scan_super_learner =  load("N:/Incubator2025_ComputationalLifeCourse/Intermediate/adl_65-75_(scan)_super_learner.pkl")

In [13]:
from sklearn.utils import resample

# Feature Names
trt_features_med_adl_65_75_dic   = gc.create_feature_df(X_med_features_adl_65_75_dic, feature_cols_outcome_med_65_75, "adl 65–75")
trt_features_cor_adl_65_75_dic   = gc.create_feature_df(X_cor_features_adl_65_75_dic, feature_cols_outcome_cor_65_75, "adl 65–75")
trt_features_scan_adl_65_75_dic   = gc.create_feature_df(X_scan_features_adl_65_75_dic, feature_cols_outcome_scan_65_75, "adl 65–75")


def run_mc_bootstrap_once_regime_fixed_model(seed):
    # Step 1: Bootstrap indices
    def bootstrap(X, y, seed):
        idx = resample(np.arange(len(X)), replace=True, n_samples=len(X), random_state=seed)
        return X[idx], y[idx]
 

    X_mob_med, y_mob_med = bootstrap(X_med_features_adl_65_75_dic, y_adl_med_65_75_dic, seed+1)
    X_dis_med, _ = bootstrap(X_med_features_disease_65_75_dic, y_adl_disease_med_65_75_dic, seed+1)

    X_mob_scan, y_mob_scan = bootstrap(X_scan_features_adl_65_75_dic, y_adl_scan_65_75_dic, seed+2)
    X_dis_scan, _ = bootstrap(X_scan_features_disease_65_75_dic, y_adl_disease_scan_65_75_dic, seed+2)

    X_mob_corp, y_mob_corp = bootstrap(X_cor_features_adl_65_75_dic, y_adl_cor_65_75_dic, seed+3)
    X_dis_corp, _ = bootstrap(X_cor_features_disease_65_75_dic, y_adl_disease_cor_65_75_dic, seed+3)

    # Step 2: Update with medoid features
    def update_with_medoids(X_dis, X_mob):
        return gc.generate_updated_list(X_dis, X_medoid_features_65_75_dic, cols_to_replace=30), \
               gc.generate_updated_list(X_mob, X_medoid_features_65_75_dic, cols_to_replace=30)
 
    X_dis_med_updated, X_mob_med_updated = update_with_medoids(X_dis_med, X_mob_med)
    X_dis_scan_updated, X_mob_scan_updated = update_with_medoids(X_dis_scan, X_mob_scan)
    X_dis_corp_updated, X_mob_corp_updated = update_with_medoids(X_dis_corp, X_mob_corp)

    # Step 3: Use pre-trained models (must be defined globally or passed in)
    # e.g., sl_dis, sl_mob, etc. are pre-trained super learners
 
    _, _, ates_med, stand_ates_med, risk_ratios_med = gc.counterfactual_y_under_X_medoid_features(
        tv_cov_model=y_adl_disease_med_super_learner,
        y_model=y_adl_med_super_learner,
        X_features_disease_65_75_dic_updated_list=X_dis_med_updated,
        X_features_outcome_65_75_dic_updated_list=X_mob_med_updated,
        tv_cov_name="dt_n_years_disease_dic",
        outcome_feature_names=trt_features_med_adl_65_75_dic.columns.tolist()
    )

    _, _, ates_cor, stand_ates_cor, risk_ratios_cor =  gc.counterfactual_y_under_X_medoid_features(
        tv_cov_model=y_adl_disease_cor_super_learner,
        y_model=y_adl_cor_super_learner,
        X_features_disease_65_75_dic_updated_list=X_dis_corp_updated,
        X_features_outcome_65_75_dic_updated_list=X_mob_corp_updated,
        tv_cov_name="dt_n_years_disease_dic",
        outcome_feature_names=trt_features_cor_adl_65_75_dic.columns.tolist()
    )

    _, _, ates_scan, stand_ates_scan, risk_ratios_scan =  gc.counterfactual_y_under_X_medoid_features(
        tv_cov_model=y_adl_disease_scan_super_learner,
        y_model=y_adl_scan_super_learner,
        X_features_disease_65_75_dic_updated_list=X_dis_scan_updated,
        X_features_outcome_65_75_dic_updated_list=X_mob_scan_updated,
        tv_cov_name="dt_n_years_disease_dic",
        outcome_feature_names=trt_features_scan_adl_65_75_dic.columns.tolist()
    )

    # Step 4: Format results
    ate_result = {"seed": seed} 
    ate_result.update(gc.flatten_ates_dict(ates_med, "Mediterranean"))
    ate_result.update(gc.flatten_ates_dict(ates_cor, "Corporatist"))
    ate_result.update(gc.flatten_ates_dict(ates_scan, "Scandinavian"))

    stand_ate_result = {"seed": seed} 
    stand_ate_result.update(gc.flatten_ates_dict(stand_ates_med, "Mediterranean"))
    stand_ate_result.update(gc.flatten_ates_dict(stand_ates_cor, "Corporatist"))
    stand_ate_result.update(gc.flatten_ates_dict(stand_ates_scan, "Scandinavian"))
    
    risk_ratio_result = {"seed": seed} 
    risk_ratio_result.update(gc.flatten_ates_dict(risk_ratios_med, "Mediterranean"))
    risk_ratio_result.update(gc.flatten_ates_dict(risk_ratios_cor, "Corporatist"))
    risk_ratio_result.update(gc.flatten_ates_dict(risk_ratios_scan, "Scandinavian"))
    

    return ate_result, stand_ate_result, risk_ratio_result



adl 65–75: Feature dataframe shape = (4706, 48)
adl 65–75: Feature dataframe shape = (7381, 48)
adl 65–75: Feature dataframe shape = (2871, 48)


In [14]:
results_adl_65_75_dic = [run_mc_bootstrap_once_regime_fixed_model(seed) for seed in range(42, 45)]
df_results_adl_65_75_dic = pd.DataFrame(results_adl_65_75_dic)


--- Medoid 0 ---
  Predicted dt_n_years_disease_dic: mean=0.001
 Potential outcome: mean=0.109

--- Medoid 1 ---
  Predicted dt_n_years_disease_dic: mean=0.005
 Potential outcome: mean=0.161

--- Medoid 2 ---
  Predicted dt_n_years_disease_dic: mean=0.003
 Potential outcome: mean=0.130

--- Medoid 3 ---
  Predicted dt_n_years_disease_dic: mean=0.007
 Potential outcome: mean=0.151

Reference medoid = 2, mean outcome = 0.130

medoid_0: ATE=-0.020, Std ATE=-0.157, RR=0.843
medoid_1: ATE=0.032, Std ATE=0.243, RR=1.243
medoid_2: ATE=0.000, Std ATE=0.000, RR=1.000
medoid_3: ATE=0.021, Std ATE=0.161, RR=1.161

--- Medoid 0 ---
  Predicted dt_n_years_disease_dic: mean=0.000
 Potential outcome: mean=0.115

--- Medoid 1 ---
  Predicted dt_n_years_disease_dic: mean=0.000
 Potential outcome: mean=0.106

--- Medoid 2 ---
  Predicted dt_n_years_disease_dic: mean=0.004
 Potential outcome: mean=0.126

--- Medoid 3 ---
  Predicted dt_n_years_disease_dic: mean=0.002
 Potential outcome: mean=0.131

Refe

In [15]:
# Fixed modles 
# Separate outcome models 

from joblib import Parallel, delayed
import pandas as pd
import time

start = time.time()

# Define number of bootstrap replicates and parallel jobs
n_bootstrap = 1000  # or any number you like
n_jobs = 8 
batch_size = 15

# Run bootstrap in parallel
bootstrap_results_adl_65_75_dic = Parallel(n_jobs=n_jobs, batch_size = batch_size)(
    delayed(run_mc_bootstrap_once_regime_fixed_model)(seed) for seed in range(42, 42 + n_bootstrap)
)

end = time.time()
print(f"Finished 1000 bootstraps in {(end - start)/60:.2f} minutes")

# Convert to DataFrame
df_bootstrap_results_adl_65_75_dic = pd.DataFrame(bootstrap_results_adl_65_75_dic)


Finished 1000 bootstraps in 9.34 minutes


In [16]:
# Separate out raw ATE, standardized ATE, and risk ratio results
raw_ate_results_adl_65_75_dic = [r[0] for r in bootstrap_results_adl_65_75_dic]
std_ate_results_adl_65_75_dic = [r[1] for r in bootstrap_results_adl_65_75_dic]
risk_ratio_results_adl_65_75_dic = [r[2] for r in bootstrap_results_adl_65_75_dic]

df_raw_ate_adl_65_75_dic = pd.DataFrame(raw_ate_results_adl_65_75_dic)
df_std_ate_adl_65_75_dic = pd.DataFrame(std_ate_results_adl_65_75_dic)
df_risk_ratio_adl_65_75_dic = pd.DataFrame(risk_ratio_results_adl_65_75_dic)

df_raw_ate_summary_adl_65_75_dic = gc.summarize_bootstrap_percentile_ci(df_raw_ate_adl_65_75_dic)
df_std_ate_summary_adl_65_75_dic = gc.summarize_bootstrap_percentile_ci(df_std_ate_adl_65_75_dic)
df_risk_ratio_summary_adl_65_75_dic = gc.summarize_bootstrap_percentile_ci(df_risk_ratio_adl_65_75_dic, risk_ratio=True)

# Save
gc.save_results_df(df_raw_ate_adl_65_75_dic, prefix = "raw_ate_df", label="adl 65–75")
gc.save_results_df(df_std_ate_adl_65_75_dic, prefix = "std_ate_df", label="adl 65–75")
gc.save_results_df(df_risk_ratio_adl_65_75_dic, prefix = "risk_ratio_df", label="adl 65–75")


gc.save_results_df(df_raw_ate_summary_adl_65_75_dic, prefix = "raw_ate_summary", label="adl 65–75")
gc.save_results_df(df_std_ate_summary_adl_65_75_dic, prefix = "std_ate_summary", label="adl 65–75")
gc.save_results_df(df_risk_ratio_summary_adl_65_75_dic, prefix = "risk_ratio_summary", label="adl 65–75")

Saved: Results\raw_ate_df_adl_65-75_all.csv
Saved: Results\std_ate_df_adl_65-75_all.csv
Saved: Results\risk_ratio_df_adl_65-75_all.csv
Saved: Results\raw_ate_summary_adl_65-75_all.csv
Saved: Results\std_ate_summary_adl_65-75_all.csv
Saved: Results\risk_ratio_summary_adl_65-75_all.csv
